# Pandas Tips & Tricks

This notebook presents various tricks to manipulate your data, which are typically non-obvious to a novice in Pandas and data science.

In [0]:
%matplotlib notebook

import time
from IPython.display import HTML, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Reading Data

### Combining Multiple CSV Files

This shows how to create a single dataframe from multiple files that share the same structure (columns).

In [0]:
import glob

files = '../data/commits*.tsv'
df = pd.concat([pd.read_csv(x, sep='\t', parse_dates=['Date']) for x in glob.glob(files)], 
               ignore_index=True)

!wc -l {files}
print('♯', len(df))

In [0]:
len(df.Author.unique()), len(pd.unique(df.Author))

## Inspecting Dataframes

Looking at the contents and metadata of your dataframes is quite important, to better understand the data they represent and then successfully transform it into the results you need.

In [0]:
# Data dimensions (rows, cols)
df.shape

In [0]:
# Data types
df.dtypes

If you look at a sample, it is often useful to transpose the data, especially when you have many columns.

In [0]:
print(df.head(2).transpose())

And then there is `describe` with some core statistics about the dataframe…

In [0]:
print(df.describe().transpose())

… and `info` with more technical information.

In [0]:
df.info()

## Writing Results
### Writing Spreadsheet Files
*TODO*

## Filtering Rows

You can use [loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) in combination with a `bool` array to select a subset of rows. That array is conveniently created by applying conditions to columns.

The first example uses regex matching…

In [0]:
print(df.loc[df.Message.str.match('altair', case=False)]
             .reindex(['Date', 'Message'], axis=1))

Another option is using simple comparison operators, e.g. `!=` like here…

In [0]:
print(df.loc[df.Author != 'jhermann']
             .reindex(['Date', 'Message'], axis=1))

Note that the condition creates a `bool` array, that then is taken by `loc[…]` to select the matching rows.

In [0]:
list(df.Author.iloc[-5:] != 'jhermann')

## Manipulating Columns

### Adding or Replacing Columns
Changing the values of a column or adding a whole new one can be done by actual assignment or by calling `assign`.

In [0]:
morecols = df.assign(Words=df.Message.str.split().apply(len))
print('Column ♯:', len(df.columns), 'vs.', len(morecols.columns))
print(morecols.head(2).iloc[:, -3:])

Using assigment is inplace and changes the dataframe.

In [0]:
morecols['Zero'] = 0
print(morecols.head(2).iloc[:, -4:])

### Renaming Columns
This one's easy, just call [rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html). Columns can be specified in various formats, like a mapping from old to new. Renaming can also be done inplace, the default is to copy.

In [0]:
print(morecols.rename(columns=dict(Message='Text')).head(1).iloc[:, -4:])

To rename all columns, just `zip` the existing names with the new ones.

In [0]:
numcols = morecols.rename(
    columns=dict(zip(morecols.columns, 
                     range(len(morecols.columns)))))
print(numcols.head(1).transpose())

To rename according to some logic, like a regex substitution or similar, provide a mapper function.

In [0]:
print(morecols.rename(mapper=str.upper, axis=1).head(1).transpose())

### Deleting Columns
*TODO*

### Selecting Columns

In [0]:
print(df[['Date', 'Message']].head(1))

## String Manipulation

The new `Day` column is just the first word out of the `Date` column. By splitting with `expand=True` two columns are created (instead of one column with tuples), so we can select the first column only and add this to the dataframe.

In [0]:
df = df.assign(Day=df.Date.astype(str).str.split(n=1, expand=True)[0])
print(df.head(1).iloc[:, -3:])

Since `Date` is a `datetime64` column, we can also use the [DatetimeProperties](http://pandas.pydata.org/pandas-docs/version/0.15.0/api.html#datetimelike-properties) accessor for day extraction.

In [0]:
df = df.assign(Day=df.Date.dt.date)
print(df.head(1).iloc[:, -3:])

## Counting

To visualize data in bar or other magnitude charts, you have to count subsets of your raw data.

In [0]:
commits_per_day = df.Day.value_counts().to_frame().sort_index()
_ = commits_per_day.plot.barh(legend=False, figsize=(5, 2))

chart_img = 'img/pandas-barh.png'
plt.savefig(chart_img)
clear_output()
HTML('<img src=\"{}?{}\"></img>'.format(chart_img, time.time()))

## Aggregation

Grouping values by one or more columns and then applying an operation to fold those values into a single scalar.

In [0]:
letters = list("Pandas")
codes = pd.DataFrame(dict(Letter=letters, Code=list(map(ord, letters))))
codes = codes.groupby('Letter').aggregate(np.sum)
print(codes.transpose())

Using `reset_index` moves the grouping column(s) from the index to ordinary columns.

In [0]:
codes = codes.reset_index()
print(codes.transpose())